# Fama–French three-factor model  


In asset pricing and portfolio management the Fama–French three-factor model is a model designed to describe stock returns.<br>
<br>
__The three factors are:__ 
1. Rm-Rf : Market risk 
2. SMB   : Small [market capitalization] Minus Big 
3. HML   : High [book-to-market ratio] Minus Low



![title](Images/fama-french-three-factor.png)

To calculate coeficients of Three Factors, you can either use __OLS regression model__ or else __calculate using formula!__ <br> 
This notebook elaborates both methods. <br>


## Method 1: Calculate Beta & Alpha using formula

###  To get the market return :

In [26]:
import yfinance as yf
import numpy as np

import plotly.express as px

In [27]:
def market_return(List, Start, End):
    """  Market Return for each stock"""    

    j = 0
    T = 44
    N = len(List)
    PORTFOLIO = np.zeros((T, N))

    for i in List:
        #print("Stock: ",i)
        stock_symbol = yf.Ticker(i)
        data = stock_symbol.history(start=Start, end=End)
        Price = (data['Open'] + data['Close']) / 2
        
        Return = Price.div(Price.shift(1)).dropna()
        Return = np.log(Return)
        
        PORTFOLIO[:,j] = Return
        j+=1

        #time_series_plot(Return)

    return PORTFOLIO


In [28]:
## To plot time series 
def time_series_plot(data):
    fig = px.line(data)
    fig.show()

In [29]:
Stock_symbols = ['JILL','ELTK','ONVO','UAVS', 'AEY','OBLN']

returns = market_return(Stock_symbols, '2019-07-01', '2019-09-04') # portfolio return

time_series_plot(returns) # plot 


# To calculate the coeficient of the factors :

In [30]:
#from EQ_01 import portfolio_return

import numpy as np 
import pandas as pd 

FAMA_FRENCH_3 = pd.read_csv("Data/F-F_Research_Data_Factors_daily.csv",names=['Mkt-RF','SMB','HML','RF'], skiprows=24516, nrows=44)
#2019-07-01 .........24517
#2019-08-30 ..........24560

FAMA_FRENCH_3.head()

,Mkt-RF,SMB,HML,RF
20190701,0.75,-0.54,-0.15,0.009
20190702,0.14,-0.81,-0.78,0.009
20190703,0.78,-0.06,-0.30,0.009
20190705,-0.10,0.16,0.90,0.009
20190708,-0.58,-0.44,0.05,0.009


In [31]:
# Converting series data to array
MKT = (FAMA_FRENCH_3['Mkt-RF'] - FAMA_FRENCH_3['RF']).to_numpy()
RF  = FAMA_FRENCH_3['RF'].to_numpy()

SMB = FAMA_FRENCH_3['SMB']
HML = FAMA_FRENCH_3['HML']


In [46]:
Stock_symbols = ['JILL','ELTK','ONVO','UAVS', 'AEY','OBLN','XRF','MLSS','MICT','SOLY','NMRD','INUV','VTGN',
                'MTSL','DXR','QADA','MYOV','BOSC','APTO','PESI','TITN','EYEG','CAPR','UROV','APDN','TTPH','ECOR','WYND','ZN',
                'OSN','HDSN','BDR','TBLT','PLM','DMRC','MOV','PVH','TLYS','CPAH','SDRL','RWLK','VEEV','OPRX','RAVE',
                'PFSW','SPWH','TMDX','DOOO','TC','NBY','WSTL','KEYS','DBI','RVLT','JASN','CNET','RVLV','REKR','SUMR',
                'MBOT','BDGE','FOLD','SRRA','IDN', 'PIXY','LYL','MHLD','NIO','SLS','NNVC', 'MYT','BYND','CREG',
                'TCCO','AAMC','OXBR','ANCN','BCRX','ALRN','RTW','LTBR','WWR','VTVT','MACK','NDRA','OTLK','MYT','BYND',
                'CREG','TCCO','AAMC','OXBR','ANCN','BCRX','ALRN','RTW','LTBR','WWR','HTHT','MACK']

In [47]:
#Market return for each stock
Portfolio_Return = market_return(Stock_symbols, '2019-06-28', '2019-08-31')

In [48]:
Portfolio_Return.shape

(44, 100)

## Step 1. Calculate Beta :


![title](Images/beta-coefficient1.jpg)

If Beta > 0 & Beta < 1 :  It implies the stock prices will move with the overall market; however, the stock prices will remain less risky and volatile.<BR>
If Beta = 1:              It has the same level of risk as to the stock market. <BR>
If Beta > 1:              It implies a higher level of risk and volatility as compared to the stock market <BR>


In [49]:
''' You can use either Time-Series Regression to calculate Beta or use the above formula '''

def get_beta(Portfolio_Return, RF, MKT):
    
    #Initializing beta with zero for each stock  
    beta = np.zeros((Portfolio_Return.shape))
    T,N = beta.shape
    
    for i in range(0,N):
        beta[:,i] = (Portfolio_Return[:,i] - RF)/ MKT
    return beta


In [50]:
beta = get_beta(Portfolio_Return, RF, MKT)
beta[:,3]

array([ 0.0328427 , -0.45367104,  0.01049866, -0.519241  ,  0.04244538,
       -0.24438474,  0.01925649,  0.30117896, -0.04703296,  0.83160643,
       -0.11627017,  0.13719981, -0.02639296,  0.04535528, -0.0373444 ,
       -0.06784118, -0.04101315, -0.04083143, -0.0325422 ,  0.02735562,
        0.2498215 ,  0.0418938 ,  0.04342531, -0.01415971,  0.01502135,
       -0.03727357, -0.11111062, -0.0142911 , -0.04405678, -0.03950524,
       -0.03159312,  0.00236727, -0.17779535, -0.03154419, -0.00617829,
        0.06669102,  0.01776859,  0.06542056, -0.01333317,  0.03152101,
       -0.10978347, -0.09698208,  0.02399159, -1.14861165])

## Step 2: Calculate Alpha :

To calculate Active return of the stocks:

* Alpha is a measure of the performance of an investment as compared to a suitable benchmark index.
  - Positive Alpha : Outperformed the overall market
  - Negative Alpha : Underperformed the overall market

#### Equation :

R = Rf + beta (Rm – Rf) + Alpha <br>

Alpha = R – Rf – beta (Rm-Rf) <br>

Where, <br>
R  -   Portfolio Return <br>
Rf -   Risk-free rate of Return <br>
Beta - Systematic risk of a Portfolio / Volatility <br>
Rm -   Market return, per a Benchmark <br>

In [51]:
def get_Alpha_Sigma(Portfolio_Return,beta,RF,MKT):
    
    
    N,T = Portfolio_Return.shape # Getting dimensions of Portfolio_Return
    
    alpha = np.zeros((N,T))   #Initializing with zero
    sigma = np.zeros((N,N,T)) #Initializing with zero
    
    for i in range(0,N):
        alpha[:,i] = Portfolio_Return[:,i] - (RF + (beta[:,i] * MKT) )  #Alpha = R – Rf – beta (Rm-Rf)
        sigma[:,:,i] = alpha[:,i] * (alpha[:,i].transpose());
    
    return alpha,sigma

In [52]:
#Portfolio return for each stock
Portfolio_Return = market_return(Stock_symbols, '2019-06-28', '2019-08-31')

beta = get_beta(Portfolio_Return, RF, MKT)

alpha,sigma =get_Alpha_Sigma(Portfolio_Return,beta,RF,MKT)


In [53]:
alpha[:,3]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.38777878e-17,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  3.46944695e-18,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  3.85486018e-19,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -8.67368355e-19,
        0.00000000e+00, -6.93889390e-18,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -8.67361738e-19,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -6.93889390e-18,  0.00000000e+00,  3.46944695e-18])

In [54]:
time_series_plot(alpha)

In [55]:
ALPHA = np.mean(alpha[1])
SIGMA = np.mean(sigma[2])

## Method 2: OLS Regression Model

In [56]:
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [67]:
X = FAMA_FRENCH_3

X1 = sm.add_constant(X) # Add a constant to the independent value

for i in range(0,5):
    y = Portfolio_Return[:,i]

    # make regression model 
    model = sm.OLS(y, X1)

    # fit model and print results
    results = model.fit()
    
    #Coeficient values: 
    print(f"{i+1}) Stock Symbol: ",Stock_symbols[i])
    print("\n Const: ",results.params[0])
    print(" Mkt-RF: ",results.params[1]," SMB: ",results.params[2]," HML: ",results.params[3]," RF: ",results.params[4])
    print("\n Summary: ")
    print(results.summary(),"\n")


1) Stock Symbol:  JILL

 Const:  -0.027291301001323383
 Mkt-RF:  0.009066652137219836  SMB:  -0.006775626007827194  HML:  -0.00351044198863826  RF:  3.092302518700798

 Summary: 
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.8533
Date:                Wed, 02 Dec 2020   Prob (F-statistic):              0.500
Time:                        16:32:30   Log-Likelihood:                 83.798
No. Observations:                  44   AIC:                            -157.6
Df Residuals:                      39   BIC:                            -148.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std er

__Pranav Khiste__ <br>
__INFO 7110__ <br>
__Northeastern University__ <br>

__Dependencies:__ <br>
pandas==1.1.4 <br>
numpy==1.18.5 <br>
plotly==4.13.0 <br>
statsmodels==0.11.1 <br>
yfinance==0.1.55 <br>